# Костин Сергей, ИУ5-65Б
## Рубежный Контроль №2, В9

Задание.
Для заданного набора данных (по Вашему варианту) постройте модели классификации или регрессии (в зависимости от конкретной задачи, рассматриваемой в наборе данных). Для построения моделей используйте методы 1 и 2 (по варианту для Вашей группы).

Оцените качество моделей на основе подходящих метрик качества (не менее двух метрик). Какие метрики качества Вы использовали и почему? Какие выводы Вы можете сделать о качестве построенных моделей? Для построения моделей необходимо выполнить требуемую предобработку данных: заполнение пропусков, кодирование категориальных признаков, и т.д.

# Импорты и либы

In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Функции

In [42]:
def descr(df, types = True):
    print("Размер датасета:", df.shape)
    if types:
        print("Столбцы и типы данных:\n", df.dtypes)
    print("Количество пропусков:\n", df.isna().sum())

# Подготовка

In [54]:
# Загрузка данных (здесь вам нужно указать путь к вашему файлу)
df = pd.read_csv('houses_to_rent.csv')
data = df.copy()
data.shape

(6080, 14)

In [55]:
# Удаление строк, содержащих 'Sem info'
data.replace('Sem info', np.nan, inplace=True)
data.replace('Incluso', 0, inplace=True)
data.dropna(inplace=True)
data.shape

(5961, 14)

In [56]:
# Обработка столбцов с ценами
price_columns = ['hoa', 'rent amount', 'property tax', 'fire insurance', 'total']
for col in price_columns:
    print(col)
    data[col] = data[col].str.replace('R$', '').str.replace(',', '').astype(float)

hoa
rent amount
property tax
fire insurance
total


In [57]:
data.replace('-', np.NaN, inplace=True)

In [58]:
# Заполнение пропусков для числовых данных
num_cols = data.select_dtypes(include=[np.number]).columns
print(num_cols)
imputer = SimpleImputer(strategy='median')
data[num_cols] = imputer.fit_transform(data[num_cols])

Index(['Unnamed: 0', 'city', 'area', 'rooms', 'bathroom', 'parking spaces',
       'hoa', 'rent amount', 'property tax', 'fire insurance', 'total'],
      dtype='object')


In [59]:
# Заполнение пропусков для категориальных данных и их кодирование
cat_cols = data.select_dtypes(include=[object]).columns
print(cat_cols)
for col in cat_cols:
    data[col] = data[col].fillna('Unknown')

label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

Index(['floor', 'animal', 'furniture'], dtype='object')


In [ ]:
# Данные уже числовые
raise

# Преобразование столбцов "можно с животными" и "есть ли мебель"
data['animal'] = data['animal'].map({'acept': 1, 'not acept': 0})
data['furniture'] = data['furniture'].map({'furnished': 1, 'not furnished': 0})

In [61]:
descr(data, False)

Размер датасета: (5961, 14)
Количество пропусков:
 Unnamed: 0        0
city              0
area              0
rooms             0
bathroom          0
parking spaces    0
floor             0
animal            0
furniture         0
hoa               0
rent amount       0
property tax      0
fire insurance    0
total             0
dtype: int64


In [62]:
data.head()

,Unnamed: 0,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa,rent amount,property tax,fire insurance,total
0,0.0,1.0,240.0,3.0,3.0,4.0,36,0,0,0.0,8000.0,1000.0,121.0,9121.0
1,1.0,0.0,64.0,2.0,1.0,1.0,1,0,1,540.0,820.0,122.0,11.0,1493.0
2,2.0,1.0,443.0,5.0,5.0,4.0,22,0,0,4172.0,7000.0,1417.0,89.0,12680.0
3,3.0,1.0,73.0,2.0,2.0,1.0,3,0,1,700.0,1250.0,150.0,16.0,2116.0
4,4.0,1.0,19.0,1.0,1.0,0.0,36,1,1,0.0,1200.0,41.0,16.0,1257.0


# Разделение

In [63]:
# Разделение данных на признаки и целевую переменную
X = data.drop(columns=['total'])
y = data['total']

In [65]:
# Масштабирование числовых данных
for col in X:
    m = X[col].max()
    X[col] = X[col] / m

In [66]:
print(X.shape)
X

(5961, 13)


,Unnamed: 0,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa,rent amount,property tax,fire insurance
0,0.000000,1.0,0.009754,0.3,0.3,0.333333,1.000000,0.0,0.0,0.000000,0.177778,0.002730,0.178730
1,0.000165,0.0,0.002601,0.2,0.1,0.083333,0.027778,0.0,1.0,0.002455,0.018222,0.000333,0.016248
2,0.000329,1.0,0.018004,0.5,0.5,0.333333,0.611111,0.0,0.0,0.018964,0.155556,0.003868,0.131462
3,0.000494,1.0,0.002967,0.2,0.2,0.083333,0.083333,0.0,1.0,0.003182,0.027778,0.000410,0.023634
4,0.000658,1.0,0.000772,0.1,0.1,0.000000,1.000000,1.0,1.0,0.000000,0.026667,0.000112,0.023634
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6075,0.999342,1.0,0.002032,0.2,0.1,0.083333,0.305556,0.0,1.0,0.001909,0.025556,0.000000,0.022157
6076,0.999506,1.0,0.003414,0.2,0.2,0.083333,0.194444,1.0,0.0,0.003491,0.064444,0.000172,0.054653
6077,0.999671,0.0,0.001951,0.1,0.1,0.000000,0.111111,0.0,1.0,0.001136,0.021111,0.000115,0.019202
6078,0.999835,1.0,0.006502,0.3,0.2,0.166667,1.000000,1.0,1.0,0.000000,0.077778,0.000683,0.078287


In [68]:
descr(X, False)

Размер датасета: (5961, 13)
Количество пропусков:
 Unnamed: 0        0
city              0
area              0
rooms             0
bathroom          0
parking spaces    0
floor             0
animal            0
furniture         0
hoa               0
rent amount       0
property tax      0
fire insurance    0
dtype: int64


In [69]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Модель

In [70]:
# Построение модели SVR
svr_model = SVR()
svr_model.fit(X_train, y_train)

# Предсказание и оценка качества SVR
y_pred_svr = svr_model.predict(X_test)
mse_svr = mean_squared_error(y_test, y_pred_svr)
r2_svr = r2_score(y_test, y_pred_svr)

print(f'SVR MSE: {mse_svr}')
print(f'SVR R2: {r2_svr}')

SVR MSE: 181671101.5156415
SVR R2: -0.017826849024859426


In [71]:
# Построение модели Gradient Boosting
gbr_model = GradientBoostingRegressor()
gbr_model.fit(X_train, y_train)

# Предсказание и оценка качества Gradient Boosting
y_pred_gbr = gbr_model.predict(X_test)
mse_gbr = mean_squared_error(y_test, y_pred_gbr)
r2_gbr = r2_score(y_test, y_pred_gbr)

print(f'Gradient Boosting MSE: {mse_gbr}')
print(f'Gradient Boosting R2: {r2_gbr}')

Gradient Boosting MSE: 5838418.803389454
Gradient Boosting R2: 0.9672897925736982


In [72]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score

In [73]:
# MAE
mae_svr = mean_absolute_error(y_test, y_pred_svr)
mae_gb = mean_absolute_error(y_test, y_pred_gbr)

# RMSE
rmse_svr = mean_squared_error(y_test, y_pred_svr, squared=False)
rmse_gb = mean_squared_error(y_test, y_pred_gbr, squared=False)

# MAPE
mape_svr = np.mean(np.abs((y_test - y_pred_svr) / y_test)) * 100
mape_gb = np.mean(np.abs((y_test - y_pred_gbr) / y_test)) * 100

# Explained Variance
ev_svr = explained_variance_score(y_test, y_pred_svr)
ev_gb = explained_variance_score(y_test, y_pred_gbr)

# Вывод метрик
print("SVR MAE:", mae_svr)
print("SVR RMSE:", rmse_svr)
print("SVR MAPE:", mape_svr)
print("SVR Explained Variance:", ev_svr)

print("Gradient Boosting MAE:", mae_gb)
print("Gradient Boosting RMSE:", rmse_gb)
print("Gradient Boosting MAPE:", mape_gb)
print("Gradient Boosting Explained Variance:", ev_gb)

SVR MAE: 3670.4423557605583
SVR RMSE: 13478.542262264176
SVR MAPE: 70.791556523741
SVR Explained Variance: 0.004481766604727766
Gradient Boosting MAE: 248.47101274557932
Gradient Boosting RMSE: 2416.282020665107
Gradient Boosting MAPE: 2.935158273006948
Gradient Boosting Explained Variance: 0.9673558549422994


c:\Users\Sargeras\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\Sargeras\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# Выводы

Для оценки качества моделей были использованы следующие метрики:

* MSE: Измеряет среднюю квадратичную ошибку между предсказанными и фактическими значениями.

* R2: Показывает долю дисперсии зависимой переменной, которая объясняется моделью. Хорошая метрика для оценки общей эффективности модели. Больше - лучше.

* Mean Absolute Error (MAE): Среднее абсолютное отклонение предсказанных значений от фактических. Метрика была выбрана, потому что она показывает **среднюю ошибку в тех же единицах**, что и исходные данные.

* Root Mean Squared Error (RMSE): Квадратный корень из среднего квадрата ошибок. RMSE был выбран, потому что он учитывает большие ошибки более строго, чем MAE.

* Mean Absolute Percentage Error (MAPE): Среднее абсолютное процентное отклонение предсказанных значений от фактических. MAPE позволяет понять относительную ошибку в процентах, что важно для бизнес-анализа.

* Explained Variance Score: Оценка доли дисперсии данных, которая объясняется моделью. Эта метрика показывает, насколько хорошо модель объясняет вариации в данных.

Выводы о качестве построенных моделей

| Metric | SVR  | GrB  | Комментарий
| ------ | ---- | ---- | ----
| MSE    | 181M | 5M   | 
| R2     | -0.01 |0.97 | SVR хуже среднего
| MAE    | 3670.44  | 248.47  |
| RMSE   | 13478.54 | 2416.28 |
| MAPE   | 70.79%   | 2.94%   |
| ExVr   | 0.004    | 0.967   |

Support Vector Regression (SVR):

    MAE: 3670.44
    RMSE: 13478.54
    MAPE: 70.79%
    Explained Variance: 0.004

Модель SVR имеет высокие значения ошибок (MAE и RMSE) и низкое значение Explained Variance. Высокое значение MAPE указывает на значительные относительные ошибки. Это свидетельствует о том, что модель SVR плохо справляется с задачей предсказания стоимости недвижимости в данном наборе данных.

Gradient Boosting (GrB):

    MAE: 248.47
    RMSE: 2416.28
    MAPE: 2.94%
    Explained Variance: 0.967

Модель Gradient Boosting демонстрирует значительно лучшие результаты. Низкие значения MAE и RMSE указывают на малые ошибки, низкое значение MAPE говорит о высокой точности в процентном выражении, а высокое значение Explained Variance свидетельствует о том, что модель хорошо объясняет вариации в данных. Эти метрики показывают, что модель Gradient Boosting является более подходящей для задачи предсказания стоимости недвижимости.